In [1]:
# In order to import from the python file without hassle, we add the current
# directory to the python path
import sys; sys.path.append(".")

Let's let the nice folks at NCBI know who we are.

In [2]:
from Bio import Entrez

Entrez.email = "mi6568@student.uni-lj.si"

# Genome assembly

## Problem 1: Eulerian paths

In this exercise, we're going to look at genome assembly using the Eulerian path algorithm you learned about in lectures. 

**a)**
Implement the `kmers` function in `helper_functions.py`. Look at the function docstrings for expected input parameters and return types. You can test your function with the provded unit tests, but you're encouraged to come up with your own tests as well. Your kmers function should be able to produce kmers of arbitrary size e.g. if we want 7-mers for the sequence *mesenchyme*, the function should return a list containing the following sequences: *mesench*, *esenchy*, *senchym*, *enchyme*. Your function should support arbitrary strides, e.g. in the previous example we used stride 1, while if we used the same string with stride 2, we would obtain the strings *mesench* and *senchym*.

**[5 points]**

**b)**
Implement the `de_bruijn_graph` function in `helper_functions.py`. Look at the function docstrings for expected input paramters and return types. You can test your function with the provded unit tests, but you're encouraged to come up with your own tests as well. De Bruijn graph construction should be done in the following way:

1. For an input list of sequences, decompose each sequence into a list of k-mers. This is done for each sequence independently.
2. For each obtained list of k-mers, add the k-mers into the De Bruijn graph as seen in the lectures.


For instance, given a string "abcde", you should return a graph corresponding to "abc" -> "bcd" -> "cde". If multiple strings are given with no overlap, your graph will contain disconnected components, e.g. for sequences "aaaa" and "bbbb", the graph should contain two disconnected components "aaa" -> "aaa" and "bbb" -> "bbb".

**[10 points]**


**c)**
Implement the `assemble_genome` function in `helper_functions.py`. Look at the function docstrings for expected input paramters and return types. You can test your function with the provded unit tests, but you're encouraged to come up with your own tests as well. Your genome assembly function should work in the following way:

1. For an input list of fragments, construct a De Bruijn graph using your `de_bruijn_graph` function you implemented in the previous exercise.
2. Find all possible Euerlian paths through the graph, i.e. all possible paths which visit each edge exactly once. Your paths should all start from a source node with in-degree zero. In case no such node exists, you should start using the first k-mer of the first sequence. You can assume you will never encounter a disconnected graph.
3. Decode your obtained paths into sequences, and return all unique genome assemblies.

**[10 points]**

In [3]:
!python test_helper_functions.py

.........
----------------------------------------------------------------------
Ran 9 tests in 0.022s

OK


## Problem 2: Minimum fragment length

We will follow the idealized example from lectures where we have perfect coverage and we have an unrealistically good sequencer that is able to sequence the entire genome in a single run. In the real world this doesn't happen, so we'll simulate what happens in the real world. We'll split this perfect sequence up into k-mers using the functions you implemented above, i.e. we will diss-assemble the genome, and then re-assemble it using the Eulerian path algorithm. Our hope is that our algorithms will be able to reconstruct the original sequence. However, as we have seen in the lectures, some assemblies do not always produce unambiguous results. Sometimes more than one reconstruction is valid! In this exercise, we want to determine the minimum fragment length necessary for unambiguous reconstruction.

**TASK:**
Take the SARS-CoV-2 spike protein nucleotide sequence and run it through your algorithm. We will pretend the sequence is a single fragment, so you can call your `assemble_genome` function with a list containing only this sequence. We want to deconstruct this sequence into k-mers and then re-assemble it. This will simulate a perfect sequencing scenario where we have perfect coverage and each k-mer is sequenced exactly once. Find all the valid assemblies of this sequence. You'll notice that there are no ambiguities for longer kmers e.g. 50-mers, but we run into them when generating shorter "reads" (kmers). What is the minimum fragment length where we can unambiguously reconstruct the initial spike protein nucleotide sequence? In other words, what is smallest k for k-mers where we obtain exactly one valid assembly from our `assemble_genome` function? Save your answer into the `minimum_fragment_length` variable. Write your observations into the `minimum_fragment_length_comments` variable.

**[10 points]**

In [4]:
import pickle
from os import path
from Bio import Entrez, SeqIO
from Bio.SeqRecord import SeqRecord
from helper_functions import *


with Entrez.efetch(db="nucleotide", rettype="gb", id="NC_045512") as handle:
    record = SeqIO.read(handle, "gb")


for feature in record.features:
    if feature.type == 'CDS':
        if 'gene' in feature.qualifiers and 'S' in feature.qualifiers['gene']:
            sars_spike_protein = str(record.seq[feature.location.start:feature.location.end])

for k in range(11, 50):
    genoms = assemble_genome([sars_spike_protein], k)
    if len(genoms) == 1 and genoms[0] == sars_spike_protein:
        print(k)
        break

12


In [5]:
minimum_fragment_length = 12
minimum_fragment_length_comments = """
It's quite interesting that we can correctly reconstruct a (long) genome with such a low k = 12. For instance well known genome 
assembler Spades uses k = [31, 127].
"""

## Problem 3: Unrealistic assumptions

In this homework, we are assuming perfect coverage, which means we assume every kmer will be sequenced from the genome exactly once. Or in our particular case, we're pretending we have a single read long read that was sequenced only once. Of course, if we actually had this in practice, we wouldn't need genome assembly algorithms, but this is a far cry from reality.

In the real world, all off these assumptions hold no water. Many things have to be taken into consideration. For example, biologists first prepare the tissue to be sequenced. Imagine someone is preparing to sequence some mouse tissue, but they forget to wear gloves and contaminate the sample with some of their own cells, and bacteria from their hands. Then the mouse sample may contain human and bacterial DNA instead of just mouse DNA. Next, the sequencing process splits the DNA molecules into short fragments at random, so we might have the same region sequenced twice, while some regions will be completely skipped. Finally, the sequencing process itself works something like this: for every nucleotide in a sequence there exists some colored reagent which binds to it. So every base, ACTG, is assigned its own color. Each nucleotide then goes under a large microscopic camera, which literally takes pictures of every nucleotide, one by one. Then, this sequence of colors is converted into a nucleotide sequence. However, sometimes reagents don't bind to the nucleotides perfectly, so this can cause ambiguities or errors in the reads.

**Question:** Write down at least **three** things we might encounter in the real world for which we have not accounted for by using kmers (you may use some issues I listed above, but think of at least one other thing we haven't mentioned yet). For each of the issues, think about how you would approach each problem. You don't need to write down a full guide, just a general overview of how we might solve each problem. Write your answer into the `unrealistic_assumptions` variable.
**[3 points]**

In [6]:
unrealistic_assumptions = """
Assumption 1:

Assumption 2:

Assumption 3:

"""

**Question:** In all of the cases we have talked about above, we pretend we are going into the genome assembly process blind. We have some DNA fragments about which we know nothing, so we have to perform assembly from the ground up. However, often times, we know something about our organisms. For example, for SARS-CoV-2, it was suspected that it was related to SARS and MERS. Think about how we could use existing genomes from related species to facilitate genome assembly. Think back to homework 2, where we looked at sequence alignment tools. Could we use some of that machinery to help with assembling the SARS-CoV-2 genome? Write down how you would approach genome assembly for SARS-CoV-2, known that it's closely related to SARS, or even the bat RaTG13. Write your answer into the `assembly_with_alignment` variable.
**[2 points]**

In [7]:
assembly_with_alignment = """
We could use our local alignment function from homework 2 to match the sequences and fix possible anomalies in the DNA fragments
"""